La función `AI_FILTER`  devuelve Verdadero o Falso para una entrada de texto o imagen determinada, lo que nos permite filtrar resultados en cláusulas `SELECT`, `WHERE` o `JOIN ... ON`.

### Sintaxis

Aplicar `AI_FILTER` a un string:

```sql
AI_FILTER( <input> )
```

Aplicar `AI_FILTER` a una sola imagen:

```sql
AI_FILTER( <predicate> , <input> )
```

Aplicar `AI_FILTER` a varias columnas con texto e imágenes, aprovechando la función `PROMPT`:

```sql
AI_FILTER( PROMPT('<template_string>',  <col_1>, … ) )
```

### Notas de uso

Para un rendimiento óptimo, debemos seguir estas pautas:

- Asegúrese de que las columnas enviadas a `AI_FILTER` no contengan valores `NULL`.
- Utilice texto sin formato en inglés para la cadena de entrada o para los argumentos de `PROMPT()`.
- Proporcione detalles para la instrucción del texto de entrada. Por ejemplo, en lugar de una declaración como "parece satisfecho", utilice "En la siguiente transcripción de soporte, el cliente parece satisfecho".
- Considere formular la entrada en forma de pregunta. Por ejemplo, "En la siguiente transcripción de soporte, ¿parece satisfecho el cliente?"

## Ejemplos

### **AI_FILTER: Texto**

In [ ]:
SELECT AI_FILTER('¿Las pirámides del sol y la luna están ubicadas en Egipto?');

In [ ]:
SELECT AI_FILTER('¿Las pirámides del sol y la luna están ubicadas en México?');

Podemos concatenar instrucciones con columnas de texto para usar esta función.

In [ ]:
-- Creamos una tabla
CREATE TABLE curso_ia.seccion_2.reviews (
    user_id STRING,
    review STRING
);

In [ ]:
-- Insertamos algunos valores en la tabla
INSERT INTO curso_ia.seccion_2.reviews (user_id, review) VALUES
('user_001', 'Increíble experiencia en el parque! Las montañas rusas son espectaculares y el personal muy amable. Definitivamente regresaré con mi familia.'),
('user_002', 'Pasamos un día fantástico. Los juegos están muy bien mantenidos y hay opciones para todas las edades. La comida también estuvo deliciosa.'),
('user_003', 'Me encantó la nueva atracción de realidad virtual. El parque está muy limpio y organizado. Los precios son justos para la calidad que ofrecen.'),
('user_004', 'Muy decepcionante mi visita. Las filas eran extremadamente largas, varios juegos estaban cerrados por mantenimiento y el personal parecía desinteresado. No creo que valga la pena el precio de entrada.');

A continuación vamos a filtrar la tabla de reviews utilizando la función `AI_FILTER` dentro de la cláusula `WHERE`.

In [ ]:
SELECT * FROM curso_ia.seccion_2.reviews
WHERE AI_FILTER(CONCAT('¿Al visitante le gustó el parque de diversiones?', review));

Para facilitar el formato de plantilla en varias columnas, Snowflake proporciona la función `PROMPT`; por ejemplo:

In [ ]:
SELECT * FROM curso_ia.seccion_2.reviews
WHERE AI_FILTER(PROMPT('¿Al visitante {0} no le gustó el parque de diversiones?', review));

## `AI_FILTER` con un JOIN

Podemos utilizar `AI_FILTER` con un JOIN para expresar la vinculación de dos tablas con un prompt en lenguaje natural que la IA pueda razonar.

Para mostrar esta funcionalidad vamos a crear dos tablas, `candidatos` y `trabajos`. Luego insertaremos algunos valores an ambas tablas y finalmente utilizaremos `AI_FILTER` en el JOIN.

In [ ]:
-- Tabla candidatos
CREATE TABLE curso_ia.seccion_2.candidatos (
    id_candidato STRING,
    cv STRING
);

INSERT INTO curso_ia.seccion_2.candidatos (id_candidato, cv) VALUES
('CAND_001', 'Desarrollador Full Stack con 5 años de experiencia en JavaScript, React, Node.js y Python. Especializado en desarrollo web y aplicaciones móviles. Certificado en AWS y experiencia con bases de datos SQL y NoSQL.'),
('CAND_002', 'Ingeniero en Datos con 4 años de experiencia en ETL, Apache Spark, Snowflake y Tableau. Experto en Python, SQL y machine learning. Experiencia trabajando con grandes volúmenes de datos en la nube.'),
('CAND_003', 'DevOps Engineer con 6 años de experiencia en automatización, CI/CD, Docker, Kubernetes y cloud computing (AWS, Azure). Conocimientos en scripting con Bash y Python, monitoreo con Grafana y Prometheus.'),
('CAND_004', 'Licenciado en Literatura con Maestría en Comunicación. 8 años de experiencia en redacción de contenidos, copywriting, corrección de estilo y gestión editorial. Especializado en marketing de contenidos y storytelling corporativo.'),
('CAND_005', 'Ejecutivo de Ventas con 7 años de experiencia en B2B y B2C. Experto en CRM (Salesforce), prospección, negociación y cierre de ventas. Historial comprobado de superar metas trimestrales en un 120% promedio.');

In [ ]:
SELECT * FROM curso_ia.seccion_2.candidatos;

In [ ]:
-- Tabla trabajos
CREATE TABLE curso_ia.seccion_2.trabajos (
    identificador_trabajo STRING,
    descripcion_trabajo STRING
);

INSERT INTO curso_ia.seccion_2.trabajos (identificador_trabajo, descripcion_trabajo) VALUES
('JOB_TEC_001', 'Desarrollador Senior - Buscamos desarrollador full stack para liderar proyectos web usando React, Node.js y bases de datos relacionales. Experiencia mínima 3 años y conocimientos en cloud computing.'),
('JOB_DATA_002', 'Analista de Datos - Posición para profesional con experiencia en ETL, SQL, Python y herramientas de visualización como Tableau o Power BI. Trabajo con big data en plataformas cloud.'),
('JOB_SALES_003', 'Gerente de Ventas Regional - Ejecutivo comercial para liderar equipo de ventas B2B. Experiencia comprobada en CRM, negociación y cumplimiento de objetivos. Sector tecnológico preferible.');

In [ ]:
SELECT * FROM curso_ia.seccion_2.trabajos;

In [ ]:
USE DATABASE curso_ia;
USE SCHEMA seccion_2;

-- Utilizar AI_FILTER en el JOIN

SELECT *
FROM candidatos
JOIN trabajos
ON AI_FILTER(PROMPT('Evalúa si este currículum {0} se ajusta a esta descripción de trabajo {1}', candidatos.cv, trabajos.descripcion_trabajo));

## **`AI_FILTER`: Imágenes**

In [ ]:
SELECT * FROM curso_ia.seccion_2.image_table
WHERE AI_FILTER(PROMPT('{0} es una imagen de perros', img));

## Limitaciones

- Las funciones de Snowflake AI no admiten la actualización incremental de tablas dinámicas.
- Las funciones de Snowflake AI no funcionan con objetos `FILE` creados a partir de archivos en los siguientes tipos de stages:
    - Internal stage con modo de cifrado `TYPE = 'SNOWFLAKE_FULL'`
    - External stage con cualquier modo de cifrado del lado del cliente:
        - `TYPE = 'AWS_CSE'`
        - `TYPE = 'AZURE_CSE'`
    - Stage de usuario
    - Stage de tabla
    - Stage con nombres entre comillas dobles